In [8]:
# load original questions TODO
n = 5
qs = []

import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

with open('../annotated_wd_data_valid_answerable.txt') as fin:
    lines = fin.readlines()
    for l in lines[:n]:
#         subject [tab] property [tab] object [tab] question
        s, p, o, q = l.strip('\n').split('\t')
        q = re.sub(chars_to_ignore_regex, '', q).lower()
        print(q)
        qs.append(q)

where was sasha vujačić born
what is a film directed by wiebke von carolsfeld
what was seymour parker gilbert's profession
in what french city did antoine de févin die 
what job does jamie hewlett have


In [4]:
# ASR FB_S2T
# for i in *.wav; do ffmpeg -i $i -ar 1600 -acodec pcm_s16le -ac 1 ./mono/$(basename $i); done
import os
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset
import soundfile as sf

model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

path = "../dev/"

# init transcripts
ts = [''] * n

# model.to("cuda")

for file in os.listdir(path):
    speech, samplerate = sf.read(path+file)
    i = int(file.split('.')[0]) - 1
    
    
    input_features = processor(
        speech,
        sampling_rate=16_000,
        return_tensors="pt"
    ).input_features  # Batch size 1

    generated_ids = model.generate(input_ids=input_features)

    transcription = processor.batch_decode(generated_ids)
    ts[i] = transcription[0].strip('</s>')

for t in ts:
    print(t)

 where was such a brachet born
 what is a hum directed by victor von carlsfinch
 what was same where parker gilbert's profession
 in what french city did anton de fivian die
 what job does jamie he'll let have


In [9]:
#### calculate metrics TODO
for i, t in enumerate(ts):
    print(qs[i], t)
    
from datasets import load_metric
wer_metric = load_metric("wer")
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=ts, references=qs)))

wer_metric = load_metric("cer")

print("Test CER: {:.3f}".format(wer_metric.compute(predictions=ts, references=qs)))

where was sasha vujačić born  where was such a brachet born
what is a film directed by wiebke von carolsfeld  what is a hum directed by victor von carlsfinch
what was seymour parker gilbert's profession  what was same where parker gilbert's profession
in what french city did antoine de févin die   in what french city did anton de fivian die
what job does jamie hewlett have  what job does jamie he'll let have
Test WER: 0.343
Test CER: 0.199
